(from https://gist.github.com/donmahallem/22134574382b7bd8a67c1550734fcfc4 . Thanks to Ryan Baumann for sharing the gist on Mastodon.)

Make sure you have GPU runtime. If you run the code below and receive the error, 'GPU device not found', click on 'Runtime' in the menu at top, 'change runtime type', >> 'select hardware acceleration' and select GPU. Then re-run the code to make sure it's running.

In [1]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

!nvidia-smi -L
#!nvidia-smi -q

Found GPU at: /device:GPU:0
GPU 0: Tesla T4 (UUID: GPU-8bfdb1f9-413a-99b7-ae92-0f61f7a7681e)


Get Meshroom and some data.

In [ ]:
# IF YOU HAVE A FOLDER WITH IMAGES IN YOUR GOOGLE DRIVE
# you can connect this notebook to your drive by using
# this code:
from google.colab import drive
drive.mount('/content/drive')

# and then your files will be available. You can click the
# > at top left and select 'files' to see the file tree.
# right-clicking on a file or folder will allow you to copy the full path
# but nb if you then paste that path into any code, you need to remove the
# `/content/` part of the path.

In [ ]:
# or you can run this cell to upload your files.

from google.colab import files

# optional upload for the meshfile

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format( name=fn, length=len(uploaded[fn])))

If you upload files, make sure to move them into their own folder. Either upload as a zip file and then use `!unzip folder.zip` or use eg. `mv *.jpg /my_dataset/`

In [ ]:
!mkdir my_dataset
!mv *.JPG my_dataset/

In [2]:
# or you can grab some data from AliceVision for the time being.
!git clone https://github.com/alicevision/dataset_buddha


Cloning into 'dataset_buddha'...
remote: Enumerating objects: 537, done.
remote: Total 537 (delta 0), reused 0 (delta 0), pack-reused 537
Receiving objects: 100% (537/537), 762.00 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (147/147), done.
Updating files: 100% (223/223), done.


In [ ]:
# get Meshroom
#!pip install jq
!apt install jq

# get Meshroom
!curl  'https://api.fosshub.com/download/' \
  -H 'authority: api.fosshub.com' \
  -H 'accept: */*' \
  -H 'accept-language: en-US,en;q=0.9' \
  -H 'content-type: application/json; charset=UTF-8' \
  -H 'origin: https://www.fosshub.com' \
  -H 'referer: https://www.fosshub.com/' \
  -H 'sec-ch-ua: "Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"' \
  -H 'sec-ch-ua-mobile: ?0' \
  -H 'sec-ch-ua-platform: "macOS"' \
  -H 'sec-fetch-dest: empty' \
  -H 'sec-fetch-mode: cors' \
  -H 'sec-fetch-site: same-site' \
  -H 'user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36' \
  --data-raw '{"projectId":"5c6fe99776dd983e3dec2213","releaseId":"6499a02acf0a39defa2f69cd","projectUri":"Meshroom.html","fileName":"Meshroom-2023.2.0-linux.tar.gz","source":"CF"}' \
  --compressed |  jq -r '.data.url' | xargs curl --output meshroom.tar.gz


!mkdir meshroom
!tar -xzf meshroom.tar.gz -C ./meshroom

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 357 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 2s (183 kB/s)
Selecting previously unselected package libonig5:amd64.
(Reading database ... 120831 files and directories currently installed.)
Preparing to unpack .../libonig5_6.9.7.1-2build1_amd64.deb ...
Unpacking libonig5:amd64 (6.9.7.1-2build1) ...
Selecting previously unselected package libjq1:am

# Meshing!

Make sure that the next line points to the correct folder, eg my_dataset if you uploaded your own, or the dataset_buddha if you used wget to get the Buddha pictures from Alicevision. (and the line below would be chanted eg `--input ./dataset_buddha/buddha --output`)

In [ ]:
!cd meshroom && ls
!mkdir ./object_out
# Force usage of GPU for feature extraction:
# https://github.com/alicevision/meshroom/issues/502
! ./meshroom/Meshroom-2023.2.0-av3.1.0-centos7-cuda11.3.1/meshroom_batch  --paramOverrides FeatureExtraction:forceCpuExtraction=0  --input ./dataset_buddha/buddha --output ./object_out


In [ ]:
# zip and download the results!
!zip -r meshobject.zip ./object_out
files.download('meshobject.zip')